In [1]:
using LinearAlgebra, Plots, Printf, SparseArrays, IterativeSolvers, LaTeXStrings

The conjugate gradient algorithm

In [2]:
function CG(A,b,eps::Float64)
   x = 0.0*b; r = b; p = r; n = 0;
   for j = 1:1e6
        n = j
        q = A*p
        a = (r'*r)/(p'*q)
        x = x + a*p
        r_old = r
        r = r - a*q
        if norm(r) < eps
            break
        end
        b = (r'*r)/(r_old'*r_old)
        p = r + b*p 
    end
    @printf("CG took %i iterations",n)
    x
end

CG (generic function with 1 method)

In [3]:
A = SymTridiagonal(fill(2.0,10),fill(-1.0,9));
b = randn(10);

In [18]:
x = CG(A,b,1e-16);
x - A\b |> norm

CG took 11 iterations

5.0292000132872715e-15

The conjugate gradient algorithm for general linear functions and inner products

In [15]:
function CG(f,b,⋄,eps::Float64)
   x = 0.0*b; r = b; p = r; n = 0;
   for j = 1:1e6
        n = j
        q = f(p)
        a = (r⋄r)/(p⋄q)
        x = x + a*p
        r_old = r
        r = r - a*q
        if sqrt(r⋄r) < eps
            break
        end
        b = (r⋄r)/(r_old⋄r_old)
        p = r + b*p
        
    end
    @printf("CG took %i iterations",n)
    x
end

CG (generic function with 2 methods)

In [16]:
f = x -> A*x
function ⋄(x,y)
    x'*y
end

⋄ (generic function with 1 method)

In [17]:
x = CG(f,b,⋄,1e-16)
A\b - x |> norm

CG took 11 iterations

5.0292000132872715e-15

Sylvester matrix equations

$$ A X + X B = C.$$

In [19]:
n = 400;
C = randn(n,n);
A = SymTridiagonal(fill(2.0,n),fill(-1.0,n-1));

In [20]:
f = x -> A*x + x*A
function ⋄(x,y)
    dot(x,y)
end

⋄ (generic function with 1 method)

In [22]:
400^2

160000

In [21]:
X = CG(f,C,⋄,1e-16)

CG took 2124 iterations

400×400 Matrix{Float64}:
  0.332625   -0.376226   -0.156727     …  -0.705067   -0.264146    -0.0730349
 -0.392785   -0.170162    0.978857        -0.0142598  -0.343852    -0.329874
 -0.271569   -0.203548    0.72816         -0.233676   -0.373069    -0.594348
  0.0093113  -0.119306    0.0964358       -1.29423    -0.9713      -0.792751
 -0.512777    0.0103784   0.558012        -1.25875    -1.30946     -1.20889
 -0.559168   -0.621303    0.569803     …  -1.91806    -1.63012     -1.01456
 -0.0653703  -0.0839631   0.256966        -1.01058    -1.0594      -0.460205
 -0.272751   -0.168074    0.362716        -0.809711   -0.76449     -0.370696
  0.250057    0.0762443   0.939809        -0.726203   -0.549268    -0.477994
 -0.172541    0.335249    0.866468        -0.0157009  -0.289189    -0.187435
 -0.296723    0.0503348   0.690393     …  -0.080591   -0.0443119   -0.431789
  0.161481   -0.037668    0.752034         0.905705    0.694625    -0.109156
  0.4814      0.474254    0.601285         0.816815 

Preconditioned CG

In [23]:
function CG(f,g,b,⋄,eps::Float64)
   x = 0.0*b; r = b; n = 0; z = g(r);  p = z;
   for j = 1:1e6
        n = j
        w = f(p)
        a = (z⋄r)/(p⋄w)
        x = x + a*p
        r_old = r
        z_old = z
        r = r - a*w
        if sqrt(r⋄r) < eps
            break
        end
        z = g(r)
        b = (z⋄r)/(z_old⋄r_old)
        p = z + b*p 
    end
    @printf("CG took %i iterations \n",n)
    x
end

CG (generic function with 3 methods)

Test in trivial case

In [24]:
A = SymTridiagonal(fill(2.0,10),fill(-1.0,9));
b = randn(10);

In [25]:
f = x -> A*x
function ⋄(x,y)
    dot(x,y)
end
g = x -> A\x

#7 (generic function with 1 method)

In [26]:
x = CG(f,g,b,⋄,1e-15)

CG took 1 iterations 


10-element Vector{Float64}:
  0.4172283266090589
  1.1752533495825577
  1.4537238432797581
  0.5246012528123585
 -0.7772136964621085
 -1.9656454411331192
 -2.013640609240777
 -1.7354477311581817
 -2.1000190246525556
 -2.3635921937097786

Back to the Sylvester equation

In [51]:
m = 49;
h = 1/(m+1)

0.02

In [52]:
C = spzeros(m,m);
C[1,:] += rand(m)
C[end,:] += rand(m)
C[:,1] += rand(m)
C[:,end] += rand(m)
C *= h^(-2)
A = SymTridiagonal(fill(2.0,m),fill(-1.0,m-1))/h^2;

In [53]:
f = X -> A*X + X*A
function ⋄(X,Y)
    h^2*dot(X,Y)
end
g = X -> (.5A+I/h^2)\((.5A+I/h^2)\X')'
#g = X -> (A+2I/h^2)\X
#g = X -> B\X

#21 (generic function with 1 method)

In [54]:
@time X1 = CG(f,C,⋄,h^2);

CG took 121 iterations  0.145612 seconds (337.57 k allocations: 40.641 MiB, 97.65% compilation time)


In [55]:
@time X2 = CG(f,g,C,⋄,h^2);

CG took 50 iterations 
  0.271468 seconds (932.90 k allocations: 74.183 MiB, 2.06% gc time, 98.46% compilation time)


In [56]:
J = kron(A |> sparse,sparse(I,m,m)) + kron(sparse(I,m,m),A |> sparse)

2401×2401 SparseMatrixCSC{Float64, Int64} with 11809 stored entries:
⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷

In [57]:
@time J\rand(m^2)

  0.002183 seconds (65 allocations: 1.711 MiB)


2401-element Vector{Float64}:
 0.0004144139409458678
 0.0008299293286228247
 0.0011327529794958762
 0.0014077540922655556
 0.0015677541695687893
 0.0017152716699408528
 0.0019170288468851003
 0.002086985209779325
 0.0022167998901468982
 0.0024269397020129733
 0.002476712277751855
 0.0024955222942473747
 0.0026119328411032404
 ⋮
 0.002536987927343962
 0.0024562076629868635
 0.002365501929854194
 0.0023153462846144104
 0.0022075013204493486
 0.0020894446063303594
 0.0019240701418308634
 0.0016893425266602704
 0.001413935481608673
 0.0011951740790384837
 0.0008801982818542597
 0.000457416908479172

Jacobi

In [58]:
Ad = SymTridiagonal(fill(0.0,m),fill(-1.0,m-1))*1/4;
Cd = C*h^2/4;

In [59]:
f_jac = X -> -Ad*X - X*Ad
X = 0*C;

In [60]:
for i = 1:150000
    Xnew = f_jac(X) + Cd
    if norm(X-Xnew) < 1e-5
        @printf("Jacobi took %i iterations",i)
        break
    end
    X = Xnew
end

Jacobi took 4180 iterations